In [1]:
#!pip install -U keras-tuner
#!pip install -U tensorflow-addons

In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import datasets

from keras_tuner import RandomSearch
from keras_tuner import Objective

import warnings
warnings.filterwarnings('ignore')

In [3]:
(X_train_reg, Y_train_reg), (X_test_reg, Y_test_reg) = datasets.boston_housing.load_data()

In [4]:
X_train_reg.shape, X_test_reg.shape, Y_train_reg.shape, Y_test_reg.shape

((404, 13), (102, 13), (404,), (102,))

In [7]:
def build_model(hyperparams):
    
    model = Sequential()
    
    model.add(layers.InputLayer(input_shape = (X_train_reg.shape[1],)))
    model.add(layers.Dense(units = hyperparams.Int("units_l1", 16, 50, step=16),
                           use_bias = hyperparams.Boolean("bias_l1"),
                           activation = hyperparams.Choice("act_l1", ["relu", "tanh"])))
    model.add(layers.Dense(units = hyperparams.Int("units_l2", 16, 50, step=16),
                           use_bias = hyperparams.Boolean("bias_l2"),
                           activation = hyperparams.Choice("act_l2", ["relu", "tanh"])))
    model.add(layers.Dense(1))

    optim = hyperparams.Choice("optimizer",["sgd","rmsprop","adam"])
    model.compile(optim, loss="mean_squared_error", metrics=["mean_squared_error"])

    return model

In [8]:
tuner1 =  RandomSearch(hypermodel=build_model,
                      objective="val_mean_squared_error",
                      #objective=Objective(name="val_mean_squared_error",direction="min"),
                      max_trials=5,
                      #seed=123,
                      project_name="Regression",
                      overwrite=True
                    )

In [9]:
tuner1.search(X_train_reg, Y_train_reg, batch_size=32, epochs=10, validation_data=(X_test_reg, Y_test_reg))

Trial 5 Complete [00h 00m 01s]
val_mean_squared_error: 83.24462127685547

Best val_mean_squared_error So Far: 69.69825744628906
Total elapsed time: 00h 00m 04s
INFO:tensorflow:Oracle triggered exit


In [10]:
best_params = tuner1.get_best_hyperparameters()

best_params[0].values

{'units_l1': 32,
 'bias_l1': False,
 'act_l1': 'tanh',
 'units_l2': 48,
 'bias_l2': True,
 'act_l2': 'relu',
 'optimizer': 'adam'}

In [11]:
best_model = tuner1.get_best_models()[0]

best_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                416       
_________________________________________________________________
dense_1 (Dense)              (None, 48)                1584      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 49        
Total params: 2,049
Trainable params: 2,049
Non-trainable params: 0
_________________________________________________________________


In [12]:
Y_test_reg_preds = best_model.predict(X_test_reg)

Y_test_reg[:5], Y_test_reg_preds[:5]

(array([ 7.2, 18.8, 19. , 27. , 22.2]),
 array([[16.0698  ],
        [24.366627],
        [24.366615],
        [24.366627],
        [24.366627]], dtype=float32))

In [13]:
tuner1.results_summary(num_trials=3)

Results summary
Results in .\Regression
Showing 3 best trials
Trial summary
Hyperparameters:
units_l1: 32
bias_l1: False
act_l1: tanh
units_l2: 48
bias_l2: True
act_l2: relu
optimizer: adam
Score: 69.69825744628906
Trial summary
Hyperparameters:
units_l1: 32
bias_l1: True
act_l1: tanh
units_l2: 48
bias_l2: True
act_l2: tanh
optimizer: sgd
Score: 77.26081085205078
Trial summary
Hyperparameters:
units_l1: 32
bias_l1: True
act_l1: relu
units_l2: 16
bias_l2: True
act_l2: tanh
optimizer: sgd
Score: 83.24462127685547
